In [54]:
%ls

literally_all_steps.ipynb


In [1]:
from simtk import openmm as mm
import numpy as np
from simtk.openmm import app, unit, XmlSerializer
from pdbfixer import PDBFixer
import parmed

from sys import stdout
import os, glob, re, urllib
import warnings
import time
warnings.filterwarnings("ignore")
os.getcwd()

'/media/matt/ext/projects/FF'

In [5]:
# can/should we combine polariazble FF with non-polarizble solvation model and vice versa
# can/should we use fb solvation models with charmm force-fields?
# how does charmm36/tip3p-pme-{b,f}.xml differ from tip3p?
# should we include the absinth implicit solvation model?
# should we add physiological ion concentration?
# how to add DrudeForce for amoeba?
# are ions being removed from calmodulin? ...yep.

ff_solv_pairs = [
    ['amber14-all.xml','amber14/tip3p.xml'],
    ['amber14-all.xml','amber14/tip3pfb.xml'],
    ['amber14-all.xml','amber14/tip4pew.xml'],
    ['amber14-all.xml','amber14/tip4pfb.xml'],
    ['amber14-all.xml','amber14/spce.xml'],
    
    ['amber14/protein.ff14SB.xml','amber14/tip3p.xml'],
    ['amber14/protein.ff14SB.xml','amber14/tip3pfb.xml'],
    ['amber14/protein.ff14SB.xml','amber14/tip4pew.xml'],
    ['amber14/protein.ff14SB.xml','amber14/tip4pfb.xml'],
    ['amber14/protein.ff14SB.xml','amber14/spce.xml'],
    
    ['amber14/protein.ff15ipq.xml','amber14/tip3p.xml'],
    ['amber14/protein.ff15ipq.xml','amber14/tip3pfb.xml'],
    ['amber14/protein.ff15ipq.xml','amber14/tip4pew.xml'],
    ['amber14/protein.ff15ipq.xml','amber14/tip4pfb.xml'],
    ['amber14/protein.ff15ipq.xml','amber14/spce.xml'],
    
    ['amberfb15.xml','amber14/tip3p.xml'],
    ['amberfb15.xml','amber14/tip3pfb.xml'],
    ['amberfb15.xml','amber14/tip4pew.xml'],
    ['amberfb15.xml','amber14/tip4pfb.xml'],
    ['amberfb15.xml','amber14/spce.xml'],
    
    ['amber99sbnmr.xml','amber14/tip3p.xml'],
    ['amber99sbnmr.xml','amber14/tip3pfb.xml'],
    ['amber99sbnmr.xml','amber14/tip4pew.xml'],
    ['amber99sbnmr.xml','amber14/tip4pfb.xml'],
    ['amber99sbnmr.xml','amber14/spce.xml'],
    
    ['charmm36.xml','charmm36/tip3p-pme-b.xml'],
    ['charmm36.xml','charmm36/tip3p-pme-f.xml'],
    ['charmm36.xml','charmm36/tip4p2005.xml'],
    ['charmm36.xml','charmm36/tip4pew.xml'],
    ['charmm36.xml','charmm36/tip5pew.xml'],
    
    ['charmm_polar_2013.xml',''],
    ['amoeba2013.xml','amoeba2013_gk.xml'],
]

# denote custom pairs (debug)
#ff_solv_pairs = [x for x in ff_solv_pairs if 'charmm' in x[0] and '4p' in x[1]]

# denote PDB IDs and labels
structures = ['1ubq'] #,'1p7f','1unc','1l2y','3cln','2vb1',]  
names = ['ubiquitin'] #,'GB3','villin','trpcage','calmodulin','HEWL']

equilibrium_steps = 1000
reporter_steps = 1000
production_steps = 10000

for ID_ndx in range(len(structures)):
    
    # create a directory for each system
    if not os.path.exists(names[ID_ndx]):
        os.mkdir(names[ID_ndx])
        
    # fetch PDB file
    urllib.request.urlretrieve('http://files.rcsb.org/download/%s.pdb'%structures[ID_ndx],
        '%s/%s.pdb'%(names[ID_ndx],structures[ID_ndx]))
        
    # load and sanitize PDB file
    pdbfile = '%s/%s.pdb'%(names[ID_ndx],structures[ID_ndx])
    fixer = PDBFixer(filename=pdbfile)        
    missing_residues = fixer.findMissingResidues()
    nonstandard_residues = fixer.findNonstandardResidues()
    fixer.replaceNonstandardResidues()
    fixer.removeHeterogens(keepWater=False)
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(7.0)
    app.PDBFile.writeFile(fixer.topology, fixer.positions, 
        open('%s/%s_fixed.pdb'%(names[ID_ndx],structures[ID_ndx]), 'w'))
    
    # loop over FF/solvent pairs
    for ff_solv in range(len(ff_solv_pairs)):
        
        # because it never hurts to
     #   try:
            # regex the FF/solvent to make a sub-directory for each FF/solvent combination
            top_name = re.sub('.*/','',re.sub('.xml','',ff_solv_pairs[ff_solv][0]))
            solv_name = re.sub('.*/','',re.sub('.xml','',ff_solv_pairs[ff_solv][1]))
            path = '%s/%s_%s' %(names[ID_ndx],top_name,solv_name)

            # create that sub-directory
            if not os.path.exists(path):
                os.mkdir(path)
            print('\nProcessing %s/%s with %s/%s'%(names[ID_ndx],structures[ID_ndx],
                top_name,solv_name))

            # load sanizitized file
            print('Creating System...')
            system_init = time.time()
            # This won't work if you want to save .GRO/.TOP files
            #fixed_structure = app.PDBFile('%s/%s_fixed.pdb'%(names[ID_ndx],structures[ID_ndx]))
            fixed_structure = parmed.load_file('%s/%s_fixed.pdb'%(names[ID_ndx],structures[ID_ndx]))
            
            # check if a separate water model is supplied
            if ff_solv_pairs[ff_solv][1]:
                forcefield = app.ForceField(ff_solv_pairs[ff_solv][0],ff_solv_pairs[ff_solv][1])
            else:
                forcefield = app.ForceField(ff_solv_pairs[ff_solv][0])
                
            modeller = app.Modeller(fixed_structure.topology, fixed_structure.positions)

            # add solvent if explicit
            if 'iamoeba.xml' not in ff_solv_pairs[ff_solv][1] and ff_solv_pairs[ff_solv][1]:
                print('adding solvent: ',forcefield)
                print(np.shape(modeller.getPositions()))
                modeller.addSolvent(forcefield,padding=1.0*unit.nanometers)

            # add drude particles or water sites for 4p/5p (if any)
            if any(x in ff_solv_pairs[ff_solv][0] for x in ['polar', 'amoeba']) or any(x in ff_solv_pairs[ff_solv][1] for x in ['4p','5p']):
                modeller.addExtraParticles(forcefield)

            # save production PDB and generate OpenMM system
            app.PDBFile.writeFile(modeller.topology, modeller.positions, 
                open('%s/%s_prod.pdb'%(names[ID_ndx],structures[ID_ndx]), 'w'))
            
            # polarizable FFs don't work (no DrudeForce?)
            if any(x in ff_solv_pairs[ff_solv][0] for x in ['charmm_polar_2013.xml', 'amoeba2013.xml']):
                system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff,
                        constraints=app.HBonds, rigidWater=True, polarization='direct', 
                        mutualInducedTargetEpsilon=0.00001)
                
            # for non-polarizable FFs
            else:     
                system = forcefield.createSystem(modeller.topology,
                        nonbondedMethod=app.PME, constraints=app.HBonds, rigidWater=False,
                        nonbondedCutoff=12*unit.angstroms, switchDistance=10*unit.angstroms)
                
            system_structure = parmed.openmm.load_topology(modeller.topology,
                                                 system,
                                                 xyz=modeller.positions)

            print('Saving GROMACS files')
            print('%s/%s.top'%(names[ID_ndx],structures[ID_ndx]))
            system_structure.save('%s/%s.top'%(names[ID_ndx],structures[ID_ndx]), overwrite=True)
            system_structure.save('%s/%s.gro'%(names[ID_ndx],structures[ID_ndx]), overwrite=True)



Processing ubiquitin/1ubq with amber14-all/tip3p
Creating System...
adding solvent:  <simtk.openmm.app.forcefield.ForceField object at 0x7f63d96c2048>
(1231, 3)
Saving GROMACS files
ubiquitin/1ubq.top
System creation took 6.772 s
Minimizing...
Equilibrating...
Running Production (0.020 ns)...
Done! Performance: 58.203 ns/day


Processing ubiquitin/1ubq with amber14-all/tip3pfb
Creating System...
adding solvent:  <simtk.openmm.app.forcefield.ForceField object at 0x7f64126c6128>
(1231, 3)
Saving GROMACS files
ubiquitin/1ubq.top
System creation took 6.347 s
Minimizing...
Equilibrating...
Running Production (0.020 ns)...
Done! Performance: 57.626 ns/day


Processing ubiquitin/1ubq with amber14-all/tip4pew
Creating System...


OSError: ubiquitin/1ubq_fixed.pdb does not exist

In [ ]:
            # write serialized system to xml
            with open('%s/%s_system.xml'%(path,structures[ID_ndx]), 'w') as f:
                f.write(XmlSerializer.serialize(system))

            # use DrudeLangevinIntegrator for polarizable FFs
            if any(x in ff_solv_pairs[ff_solv][0] for x in ['charmm_polar_2013.xml', 'amoeba2013.xml']):
                integrator = mm.DrudeLangevinIntegrator(300*unit.kelvin, 1.0/unit.picosecond,
                    1*unit.kelvin, 1.0/unit.picosecond, 2.0*unit.femtoseconds)

            # use LangevinIntegrator for non-polarizable FFs
            else:
                integrator = mm.LangevinIntegrator(300*unit.kelvin, 1.0/unit.picoseconds, 
                    2.0*unit.femtoseconds)


            integrator.setConstraintTolerance(0.00001)
            with open('%s/%s_integrator.xml'%(path,structures[ID_ndx]),'w') as f:
                f.write(XmlSerializer.serialize(integrator))
                
            
            #platform = mm.Platform.getPlatformByName('CUDA')
            #properties = {'CudaDeviceIndex': '0', 'CudaPrecision': 'mixed'} #'0,1':double
            simulation = app.Simulation(system_structure.topology, system, integrator) #, platform, properties)
            simulation.context.setPositions(system_structure.positions)
            
            system_final = time.time()
            elapsed_time = system_final - system_init
            print('System creation took %.3f s' %elapsed_time)
            
            # run dynamics
            print('Minimizing...')
            simulation.minimizeEnergy()

            print('Equilibrating...')
            simulation.context.setVelocitiesToTemperature(300*unit.kelvin)
            simulation.step(equilibrium_steps)
            state = simulation.context.getState(getPositions=True, getVelocities=True, getForces=True, getEnergy=True, getParameters=True)
            with open('%s/%s_state.xml'%(path,structures[ID_ndx]),'w') as f:
                f.write(XmlSerializer.serialize(state))
            
            production_ns = production_steps / 500000.
            print('Running Production (%.3f ns)...'%production_ns)
            production_init = time.time()
            simulation.reporters.append(app.DCDReporter('%s/%s_trajectory.dcd'%(path,structures[ID_ndx]), reporter_steps))
            simulation.reporters.append(app.StateDataReporter('%s/%s_data.tsv'%(path,structures[ID_ndx]), reporter_steps, step=True, 
                potentialEnergy=True, temperature=True, progress=True, remainingTime=True, 
                speed=True, totalSteps=reporter_steps, separator='\t'))
            simulation.reporters.append(app.PDBReporter('%s/%s_state.pdb'%(path,structures[ID_ndx]),production_steps-1))
            simulation.step(production_steps)
            
            # evaluate performance
            production_final = time.time()
            elapsed_time = production_final - production_init
            performance = (production_ns) / (elapsed_time / 3600 / 24)
            print('Done! Performance: %.3f ns/day\n' %performance)
            
       # except Exception as e:
       #     print(e)
       #     pass

In [46]:
def foo():
    for i in range(2):
        print('foo')
        yield i

def bar():
    for i in range(2,4):
        print('bar')
        return [i]

a = foo()
b = bar()


bar
